In [1]:
#Imports
import logging
import os, random, time, sys
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers
import ipdb
import keras

sys.path.append('../../')
import feedforward_robust as ffr

sys.path.append('../../utils/')
from utils.mnist_corruption import *
from utils.utils_models import *
from utils.utils_analysis import *
from utils.utils_feedforward import *

%load_ext autoreload
%autoreload 2

#Read the counter
ctr_file = "../counter.txt"
f = open(ctr_file, 'r')
counter = f.readline()
f.close()

counter = 1 + int(counter)
f = open(ctr_file,'w')
f.write('{}'.format(counter))
f.close()
logfile = "../logs/results_" + str(counter) + ".log"

logger = logging.getLogger("robustness")
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler(logfile)
fh.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)
logger.addHandler(logging.StreamHandler())

Using TensorFlow backend.
W0610 18:32:41.013946 4613092800 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/cleverhans/utils_tf.py:341: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.



# Get Fashion MNIST Data

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train/255
x_test = x_test/255

In [4]:
#One hot the labels
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)                                                                                         
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
x_train_master, y_train_master = x_train[:30000], y_train[:30000]
x_train_slave, y_train_slave = x_train[30000:], y_train[30000:]

In [6]:
#Flatten everything
x_train_master_flat, input_shape = flatten_mnist(x_train_master) 
x_train_slave_flat, _ = flatten_mnist(x_train_slave)
x_test_flat, _  = flatten_mnist(x_test)

# Train Master

In [7]:
#Configurations
eps_train = 0.1                                                                                                                            
eps_test = 0.1                                                                                                                             
tensorboard_dir = "../tb/"                                                                                                                
weights_dir = "../weights/"                                                                                                               
load_weights = False                                                                                                              
load_counter = 234                                                                                                            
sigma = tf.nn.relu                                                                                                                         
#epochs, reg, lr = 300, 0.00, 1e-4    
epochs, reg, lr = 10, 0.00, 1e-3                                                                                                          
pgd_eta, pgd_num_iter = 1e-2, 50

In [8]:
#Setup - Dataset stuff
sess = tf.Session()
hidden_sizes = [64, 64, 32]
dataset = ((x_train_master_flat, y_train_master), (x_test_flat, y_test))

scope_name = "teacher_student_fashion"
if not load_weights:
    with tf.variable_scope(scope_name, reuse = False) as scope:

        logdir = tensorboard_dir + str(counter)

        #Create model
        writer = tf.summary.FileWriter(logdir)
        model = ffr.RobustMLP(input_shape, hidden_sizes, num_classes, writer = writer, scope = scope_name, logger = logger, sigma = sigma)
        sess.run(tf.global_variables_initializer())
        print("Created model successfully. Now going to train")
    
        #Train model
        model.fit(sess, x_train_master_flat, y_train_master, training_epochs = epochs, reg_op = reg , lr = lr)
        
        """
        #Save weights
        weights = tf.trainable_variables()
        #weights = model.get_weights()[0] + model.get_weights()[1]
        saver = tf.train.Saver(weights)
        weights_path = saver.save(sess, weights_dir + "model_" + str(counter) + ".ckpt")
        print("Saved model at %s"%weights_path)
        """
        
        #Test model - regular, fgsm adv, pgd adv
        loss_reg, acc_reg = model.evaluate(sess, x_test_flat, y_test)
        print("----Regular test accuracy and loss ----")
        print((loss_reg, acc_reg))
        
        loss_fgsm, acc_fgsm = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = False)
        print("----FGSM test accuracy and loss ----")
        print((loss_fgsm, acc_fgsm))
        
        loss_pgd, acc_pgd = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = True, eta=pgd_eta, num_iter = pgd_num_iter)
        print("----PGD test accuracy and loss ----")
        print((loss_pgd , acc_pgd))

Initialized instance variables of the robust model class
I0610 18:32:41.989053 4613092800 feedforward_robust.py:32] Initialized instance variables of the robust model class
W0610 18:32:41.990164 4613092800 deprecation_wrapper.py:119] From ../../feedforward_robust.py:36: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

Created placeholders for x and y
I0610 18:32:41.992779 4613092800 feedforward_robust.py:38] Created placeholders for x and y
W0610 18:32:41.994800 4613092800 deprecation_wrapper.py:119] From ../../utils/utils_feedforward.py:34: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0610 18:32:41.995407 4613092800 deprecation_wrapper.py:119] From ../../utils/utils_feedforward.py:34: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0610 18:32:41.996784 4613092800 deprecation_wrapper.py:119] From ../../utils/utils_feedforward.py:36: The name tf.get_variable is deprecat

Created model successfully. Now going to train


W0610 18:32:42.681906 4613092800 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py:193: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.
Epoch: 0001    cost: 0.596375283 
I0610 18:32:44.054145 4613092800 feedforward_robust.py:714] Epoch: 0001    cost: 0.596375283 
Accuracy on batch: 0.812500
I0610 18:32:44.055294 4613092800 feedforward_robust.py:715] Accuracy on batch: 0.812500
Epoch: 0002    cost: 0.418322287 
I0610 18:32:45.059883 4613092800 feedforward_robust.py:714] Epoch: 0002    cost: 0.418322287 
Accuracy on batch: 0.843750
I0610 18:32:45.061259 4613092800 feedforward_robust.py:715] Accuracy on batch: 0.843750
Epoch: 0003    cost: 0.375494492 
I0610 18:32:46.022912 4613092800 feedforward_robust.py:714] Epoch: 0003    cost: 0.375494492 
Accuracy on batch: 0.812500
I0610 18:32:46.024443 4613092800 fe

----Regular test accuracy and loss ----
(0.39057168, 0.8655)
----FGSM test accuracy and loss ----
(9.383926, 0.0323)
iteration: 0
loss 0.873708
iteration: 1
loss 1.650008
iteration: 2
loss 2.705885
iteration: 3
loss 3.958185
iteration: 4
loss 5.308160
iteration: 5
loss 6.700696
iteration: 6
loss 8.099629
iteration: 7
loss 9.472996
iteration: 8
loss 10.794743
iteration: 9
loss 12.053759
iteration: 10
loss 13.247781
iteration: 11
loss 14.371333
iteration: 12
loss 15.419714
iteration: 13
loss 16.387943
iteration: 14
loss 17.276617
iteration: 15
loss 18.082125
iteration: 16
loss 18.800011
iteration: 17
loss 19.423288
iteration: 18
loss 19.951204
iteration: 19
loss 20.376629
iteration: 20
loss 20.734335
iteration: 21
loss 21.048632
iteration: 22
loss 21.324717
iteration: 23
loss 21.568668
iteration: 24
loss 21.781284
iteration: 25
loss 21.966791
iteration: 26
loss 22.125935
iteration: 27
loss 22.265978
iteration: 28
loss 22.387779
iteration: 29
loss 22.495520
iteration: 30
loss 22.589088
it

This is to confirm that attack does not violate constraints
I0610 18:32:58.179039 4613092800 feedforward_robust.py:494] This is to confirm that attack does not violate constraints
Should be no more than eps
I0610 18:32:58.180274 4613092800 feedforward_robust.py:495] Should be no more than eps
0.10000005352730845
I0610 18:32:58.225305 4613092800 feedforward_robust.py:496] 0.10000005352730845


----PGD test accuracy and loss ----
(23.34275, 0.0015)


## Generate data for slave

In [9]:
with tf.variable_scope(scope_name, reuse = False) as scope:
        loss_reg, acc_reg = model.evaluate(sess, x_test_flat, y_test)
        print("----Regular test accuracy and loss ----")
        print((loss_reg, acc_reg))
        
        z_train_slave = model.get_prediction(sess, x_train_slave_flat)
        z_test_slave = model.get_prediction(sess, x_test_flat)

Model was evaluated on benign data
I0610 18:32:58.327879 4613092800 feedforward_robust.py:640] Model was evaluated on benign data


----Regular test accuracy and loss ----
(0.39057168, 0.8655)


In [10]:
z_train_slave[0]

array([  0.21138227,   1.0028282 ,  -3.4137535 ,   8.74303   ,
        -0.61716294, -20.783663  ,  -2.7557034 , -21.851446  ,
        -6.4539423 , -11.170397  ], dtype=float32)

# Train slave

In [13]:
#Setup - Dataset stuff
def slave_training():
    tf.reset_default_graph()
    sess = tf.Session()
    hidden_sizes = [64, 64, 32]
    dataset = ((x_train_slave_flat, z_train_slave), (x_test_flat, y_test))

    scope_name = "teacher_student_fashion"
    if not load_weights:
        with tf.variable_scope(scope_name, reuse = tf.AUTO_REUSE) as scope:

            logdir = tensorboard_dir + str(counter)

            #Create model
            writer = tf.summary.FileWriter(logdir)
            model = ffr.RobustMLP(input_shape, hidden_sizes, num_classes, writer = writer, scope = scope_name, logger = logger, sigma = sigma, classification = False)
            sess.run(tf.global_variables_initializer())
            print("Created model successfully. Now going to train")

            #Train model
            model.fit(sess, x_train_slave_flat, z_train_slave, training_epochs = epochs, reg_op = reg , lr = lr)

            """
            #Save weights
            weights = tf.trainable_variables()
            #weights = model.get_weights()[0] + model.get_weights()[1]
            saver = tf.train.Saver(weights)
            weights_path = saver.save(sess, weights_dir + "model_" + str(counter) + ".ckpt")
            print("Saved model at %s"%weights_path)
            """

            #Test model - regular, fgsm adv, pgd adv
            loss_real_reg, acc_real_reg = model.evaluate(sess, x_test_flat, z_test_slave)
            print("----Regular test accuracy and loss ----")
            print((loss_real_reg, acc_real_reg))
            
            loss_class_reg, acc_reg = model.evaluate(sess, x_test_flat, y_test)
            print("----Real test accuracy and loss comparing to teacher ----")
            print((loss_class_reg, acc_reg))

            loss_fgsm, acc_fgsm = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = False)
            print("----FGSM test accuracy and loss ----")
            print((loss_fgsm, acc_fgsm))

            loss_pgd, acc_pgd = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = True, eta=pgd_eta, num_iter = pgd_num_iter)
            print("----PGD test accuracy and loss ----")
            print((loss_pgd , acc_pgd))
            
            return loss_reg, 

Initialized instance variables of the robust model class
I0610 18:36:27.793619 4613092800 feedforward_robust.py:32] Initialized instance variables of the robust model class
Created placeholders for x and y
I0610 18:36:27.798917 4613092800 feedforward_robust.py:38] Created placeholders for x and y
Created layers and tensor for logits
I0610 18:36:27.881392 4613092800 feedforward_robust.py:42] Created layers and tensor for logits
Added accuracy computation to the graph
I0610 18:36:27.888337 4613092800 feedforward_robust.py:46] Added accuracy computation to the graph
Added MSE loss computation to the graph
I0610 18:36:27.894678 4613092800 feedforward_robust.py:58] Added MSE loss computation to the graph
Model graph was created
I0610 18:36:27.895742 4613092800 feedforward_robust.py:60] Model graph was created


Created model successfully. Now going to train


Epoch: 0001    cost: 1354.812823447 
I0610 18:36:29.512566 4613092800 feedforward_robust.py:714] Epoch: 0001    cost: 1354.812823447 
Accuracy on batch: 0.875000
I0610 18:36:29.514161 4613092800 feedforward_robust.py:715] Accuracy on batch: 0.875000
Epoch: 0002    cost: 330.429121703 
I0610 18:36:30.409995 4613092800 feedforward_robust.py:714] Epoch: 0002    cost: 330.429121703 
Accuracy on batch: 0.906250
I0610 18:36:30.411363 4613092800 feedforward_robust.py:715] Accuracy on batch: 0.906250
Epoch: 0003    cost: 239.747939445 
I0610 18:36:31.324705 4613092800 feedforward_robust.py:714] Epoch: 0003    cost: 239.747939445 
Accuracy on batch: 0.906250
I0610 18:36:31.325858 4613092800 feedforward_robust.py:715] Accuracy on batch: 0.906250
Epoch: 0004    cost: 191.715804565 
I0610 18:36:32.218352 4613092800 feedforward_robust.py:714] Epoch: 0004    cost: 191.715804565 
Accuracy on batch: 0.937500
I0610 18:36:32.219813 4613092800 feedforward_robust.py:715] Accuracy on batch: 0.937500
Epoch:

----Regular test accuracy and loss ----
(3963251.5, 0.8538)
----FGSM test accuracy and loss ----
(20922640.0, 0.8099)
iteration: 0
loss 0.863301
iteration: 1
loss 1.551709
iteration: 2
loss 2.482691
iteration: 3
loss 3.588220
iteration: 4
loss 4.774253
iteration: 5
loss 5.984824
iteration: 6
loss 7.195096
iteration: 7
loss 8.386842
iteration: 8
loss 9.547078
iteration: 9
loss 10.660930
iteration: 10
loss 11.718728
iteration: 11
loss 12.710370
iteration: 12
loss 13.630534
iteration: 13
loss 14.476028
iteration: 14
loss 15.243172
iteration: 15
loss 15.929166
iteration: 16
loss 16.530148
iteration: 17
loss 17.042925
iteration: 18
loss 17.465288
iteration: 19
loss 17.793537
iteration: 20
loss 18.060768
iteration: 21
loss 18.293888
iteration: 22
loss 18.499531
iteration: 23
loss 18.679527
iteration: 24
loss 18.833565
iteration: 25
loss 18.966845
iteration: 26
loss 19.081621
iteration: 27
loss 19.180250
iteration: 28
loss 19.265211
iteration: 29
loss 19.340038
iteration: 30
loss 19.403435
it

This is to confirm that attack does not violate constraints
I0610 18:36:43.594653 4613092800 feedforward_robust.py:494] This is to confirm that attack does not violate constraints
Should be no more than eps
I0610 18:36:43.599056 4613092800 feedforward_robust.py:495] Should be no more than eps
0.10000005352730845
I0610 18:36:43.645316 4613092800 feedforward_robust.py:496] 0.10000005352730845


----PGD test accuracy and loss ----
(9183898.0, 0.0041)
